# PyironFlow: a visual programming environment for pyiron_workflows

Author: Jörg Neugebauer

Date: Aug. 16, 2024

Key concepts:
- Strict separartion between logical/programmatic part (handled by pyiron_workflows) and gui (handled by a combination of reactflow, a react javascript tool, and ipywidgets to connect to the jupyter ecosystem)
- The communication between python and .jsx is handled via traitlets
- Graphical and programmatic representation of a workflow, described by its nodes and edges, is fully echangable. The two representations can be seemlessly interchanged, i.e. you can start writing the workflow as code and then transfer it to the gui to extend or run it 

### Setup configuration

#### Include the pyiron_node library into PYTHONPATH (modules need to be importable)

In [1]:
%config IPCompleter.evaluation='unsafe'

import sys
from pathlib import Path
sys.path.insert(0, str(Path(Path.cwd()).parent) + '/pyiron_nodes')

#### When creating a new conda environment or react widget install the following tools 

Note: You have to install via conda-forge: 
- nodejs
- esbuild
- anywidget

Uncomment the line below to install the following modules (.jsx side)

In [2]:
# !npm install react react-dom @xyflow/react @anywidget/react

#### Run the following line whenever you modify a file in the js/ (javascript) directory 

In [3]:
!npx esbuild js/widget.jsx --minify --format=esm --bundle --outdir=static

⠙
  static/widget.js   308.0kb
  static/widget.css   15.1kb

⚡ Done in 70ms
⠙

## Simple Demonstrator (How to use *anywidget* to build a reactflow widget)

In [4]:
import anywidget
import pathlib
import traitlets
import os
import json

In [20]:
class ReactFlowWidget(anywidget.AnyWidget):
    path = pathlib.Path(os.getcwd()) / 'static' 
    _esm = path / "widget.js"
    _css = path / "widget.css"
    nodes = traitlets.Unicode('[]').tag(sync=True)
    edges = traitlets.Unicode('[]').tag(sync=True)
    commands = traitlets.Unicode('[]').tag(sync=True)

### Create an empty reactflow window

In [21]:
w = ReactFlowWidget()
w

ReactFlowWidget()

#### Create some nodes and load them into the widget (to see the effect go back the the frame above)

Note: We have to convert the nodes dictionary to json to load it

In [25]:
nodes = [
    {
      'id': '1',
      'data': { 'label': 'Hello' },
      'position': { 'x': 0, 'y': 0 },
      'type': 'input',
      'sourcePosition': 'right',
    },
    {
      'id': '2',
      'data': { 'label': 'message' },
      'position': { 'x': 250, 'y': 100 },
      'type': 'output',
      'targetPosition': 'left',      
    },
    {
      'id': '3',
      'data': { 'label': 'my_node_1', 
               'source_labels': ['a', 'b'],
               'target_labels': ['in1', 'in_2', 'in_3'],
               'target_values': ['1', None, 2],
              },
      'position': { 'x': 0, 'y': 100 },
      'type': 'customNode',
      'style': {
              'border': '1px black solid',
              'padding': 5,
              'background': '#999',
              'border-radius': '10px',
              'width': '200px',
            },    
      'targetPosition': 'left',  
      'sourcePosition': 'right',  
    },
    {
      'id': '4',
      'data': { 'label': 'my_node_2', 
               'source_labels': ['a', 'c', '12'],
               'target_labels': ['in1', 'in_2'],
               'target_values': ['1', None],
              },
      'position': { 'x': 300, 'y': 160 },
      'type': 'customNode',
      'style': {
              'border': '1px black solid',
              'padding': 20,
              'background': '#1999',
            },    
      'targetPosition': 'left',  
      'sourcePosition': 'right',  
    }
]

nodes_json = json.dumps(nodes)
w.nodes = nodes_json

Play a bit with the widget. Move the nodes, connect the node handles. You can get all the changes you did graphicall via the nodes and edges property of the widget:

In [8]:
json.loads(w.edges)

[]

## PyironFlow Gui

#### Definition via workflow

In [9]:
!npx esbuild js/widget.jsx --minify --format=esm --bundle --outdir=static

⠙
  static/widget.js   308.0kb
  static/widget.css   15.1kb

⚡ Done in 56ms
⠙

In [10]:
%config IPCompleter.evaluation='unsafe'

import sys
from pathlib import Path
sys.path.insert(0, str(Path(Path.cwd()).parent) + '/pyiron_nodes')

### Create two example workflows (to compute elastic constants and an empty one)

In [11]:
from pyiron_workflow import Workflow   

Workflow.register("node_library.atomistic", "atomistic") 

wf = Workflow('compute_elastic_constants')
wf.engine = wf.create.atomistic.engine.ase.M3GNet()
wf.bulk = wf.create.atomistic.structure.build.bulk('Pb', cubic=True)
wf.elastic = wf.create.atomistic.property.elastic.elastic_constants(structure=wf.bulk, engine=wf.engine) #, parameters=parameters)
# out = elastic.pull()

wf_new = Workflow('new_workflow')

In [12]:
from python.pyironflow import PyironFlow

pf = PyironFlow([wf, wf_new])
pf.gui

### Get the grapically build or modified workflow

In [13]:
wf = pf.get_workflow()
wf.bulk.outputs.channel_dict['structure'].value

NOT_DATA

In [14]:
pf.wf_widgets[1].gui.nodes

'[]'

### You can return the workflow back to the gui 

Note: 
- all the nodes and connections are reproduced
- the node positions are presently not stored, you have to place them manually

In [15]:
wf = pf.get_workflow()
pf2 = PyironFlow([wf])
pf2.gui

#### Get the nodes of the graphically represented workflow (remove the semicolon)

In [16]:
import json
json.loads(pf.wf_widgets[0].gui.nodes);

#### Run the workflow manually (you can also do this in the gui)

In [17]:
out = wf.run()

/Users/joerg/miniforge3/envs/py12/lib/python3.12/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/Users/joerg/miniforge3/envs/py12/lib/python3.12/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/Users/joerg/miniforge3/envs/py12/lib/python3.12/site-packages/matgl/apps/pes.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.

#### Open the log window for debugging

In [18]:
pf.out_log

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

TODO:
  - Include values at ports (if simple data structures)
  - Tree view for new nodes
  - build wf from graph
  - add top buttons: 'run' (entire workflow)
  - automatic arrangement of nodes
  - Allow editing input channels (e.g. chemical element)
  - Bugs:
      - node with multiple outputs cannot be jsonified (get_import_path(node) fails)

### Ideas, concepts

The follwoing ipywidget may be useful (e.g. to replace the tabs)

In [19]:
from ipywidgets import TagsInput
TagsInput(
    value=['pizza', 'fries', '+'],
    # allowed_tags=['pizza', 'fries', 'tomatoes', 'steak'],
    allow_duplicates=False
)

TagsInput(value=['pizza', 'fries', '+'], allow_duplicates=False)